# Histfactory: A primer

### A first guide to using histfactory.

In [1]:
import ROOT
%jsroot on

Welcome to JupyROOT 6.10/06


## Introduction

RooFit is a highly flexible system to produce statistical models from a variety of probability density functions (pdfs) in various forms. The `HistFactory` is a tool to build (somewhat restricted) pdfs from simple ROOT histograms. The histograms form templates that can be used by the tool for build complex pdfs from these modular components. 

The resulting pdf is stored in a RooWorkspace which can be saved to a read using the other code snippets in this workbook.

## Preliminaries

Let us consider the first case of a single channel with one signal and one background contribution and no systematics based on the discriminating variable is $x$. While we will not continue with this notation, let us start with the familiar convention where the number of signal events is denoted as $S$ and the number of background events as $B$. Similarly, denote the signal and background “shapes” as $f_S(x)$ and $f_B(x)$ and note the these are proba- bility density functions normalized so that   $\int dxf(x) = 1$. It is common to introduce a “signal strength” parameter $\mu$ such that $\mu$ = 0 corresponds to the background-only hypothesis and $\mu$ = 1 corresponds to the nominal signal+background hypothesis. This continuous parameter $\mu$ is our parameter of interest.

Now we ask what the probability model is for obtaining n events in the data where the discriminating variable for event $e$ has a value $x_e$; thus the full dataset will be denoted {$x_1 . . . x_n $}. First one must include the Poisson probability of obtaining n events when $\mu S + B$ are expected. Secondly, one must take into account the probability density of obtaining $x_e$ based on the relative mixture $f_S(x)$ and $f_B(x)$ for a given value of $\mu$. Putting those two ingredients together one obtains what statisticians call a “marked Poisson model”:

$$
\mathcal{P}(\{x_1...x_n\}|\mu)=\mathrm{Pois}(n|\mu S+B) \left[\prod^n_{e=1}\frac{\mu S f_S(x_e) + B f_B(x_e)}{\mu S + B}\right]
$$

If one imagines the data as being fixed, then this equation depends on \mu and is called the likelihood function $L(\mu)$. Simply taking the logarithm of the equation above and remembering that Pois($n|\nu$) = $\nu^ne^{-\nu}/n!$ gives us a familiar formula referred to by physicists as an “extended maximum likelihood fit” :

$$
\ln L(\mu)= −n\ln(\mu S+B)+(\mu S+B)+ \ln n!− \left[\sum^n_{e=1}\frac{\mu S f_S(x_e) + B f_B(x_e)}{\mu S + B}\right]
$$
$$
= (\mu S+B)+ \ln n!− \sum^n_{e=1}\ln[\mu S f_S(x_e) + B f_B(x_e)]
$$
Since HistFactory is based on histograms, it is natural to think of the binned equivalent of the probability model above. Denoted the signal and background histograms as $\nu_b^\mathrm{sig}$ and $\nu_b^\mathrm{bkg}$, where $b$ is the bin index and the histogram contents correspond to the number of events expected in the data. We can relate the bin $\nu_b$ and the shape $f(x)$ via

$$
fS(x_e) = \frac{\nu_{b_e}^\mathrm{sig}}{S\Delta b_e} \quad \mathrm{and}\quad fB(x_e) = \frac{\nu_{b_e}^\mathrm{bkg}}{B\Delta b_e}
$$

where $b_e$ is the index of the bin containing $x_e$ and $\Delta b_e$ is the width of that same bins. **Note**, *because the $f(x)$ are normalized to unity we have $S=\sum_b\nu^\mathrm{sig}_b$ and $B=\sum_b\nu^\mathrm{bkg}_b$.*

Formally one can either write the probability model in terms of a product over Poisson distributions for each bin of the histogram, or one can also continue to use the unbinned expression above recognizing that the shapes $f(x)$ look like histograms (ie. they are discontinuous at the bin boundaries and constant between them). Technically, the `HistFactory` makes a model that looks more like the unbinned expression with a single RooAbsPdf that is “extended” with a discontinuous shape in $x$. Nevertheless, it can be more convenient to express the model in terms of the individual bins. Then we have   

$$
\mathcal{P}(n_b|\mu)=\mathrm{Pois}(n_\mathrm{tot}|\mu S+B) \left[\prod_{e\in \mathrm{bins}}\frac{\mu \nu_b^\mathrm{sig}+\nu_b^\mathrm{bkg}}{\mu S + B}\right] = \mathcal{N}_\mathrm{comb}\prod_{e\in \mathrm{bins}}\mathrm{Pois}(n_b|\mu\nu_b^\mathrm{sig}+\nu_b^\mathrm{bkg})
$$

where $n_b$ is the data histogram and $\mathcal{N}_\mathrm{comb}$ is a combinatorial factor that can be neglected since it is constant. Similarly, denote the data histogram is $n_b$.

## Generalizations and Use-Cases
Based on the discussion above, we want to generalize the model in the following ways:
 * Ability to include multiple signal and background samples
 * Ability to include unconstrained scaling of the normalization of any sample (as was done with $\mu$)
 * Ability to parametrize variation in the normalization of any sample due to some systematic effect
 * Ability to parameterize variations in the shape of any sample due to some systematic effect
 * Ability to include bin-by-bin statistical uncertainty on the normalization of any sample
 * Ability to incorporate an arbitrary contribution where each bin’s content is parametrized individually
 * Ability to combine multiple channels (regions of the data defined by disjoint event selections) and correlate the parameters across the various channels
 * Ability to use the combination infrastructure to incorporate control samples for data- driven background estimation techniques
 * Ability to reparametrize the model

Hopefully each of these use-cases will be demonstrated below.

## Example: $H \to \gamma \gamma$ Fit

This is an example of creating a model in order to determine the number of Higgs signal events in a sample of 2-photon invariant mass.

### Data
We have a data set consists of ~ 30000 di-photon invariant mass values from a tetx file (**Hgg.txt**) which should be in the current directory. 

In [2]:
tree = ROOT.TTree("tree","tree")
nevt = tree.ReadFile("Hgg.txt","x")
if (nevt <= 0):
    print "Error reading data from input file"

print "Read ", nevt, " events from the file "

Read  30770  events from the file 


In [3]:
data_hist = ROOT.TH1D("data_hist","Invariant Mass distribution",100,110,160)
tree.Draw("x >> data_hist")

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


### Templates

We make now the RooFit model. We assume a Gaussian distribution for the signal and an exponential distribution for the background as following: 

$$ P(x | \mu , \nu ) = n_{sig} \times G(x | M , \sigma) + n_{bkg_1} \times E(x|a_1) + n_{bkg_2} \times E(x|a_2)$$

where $G (x | M , \sigma)$ is the Gaussian distribution for the signal and $E(x|a_1)$ is the exponential distribution describing the background. 

$$E(x|a) = \frac{1}{a} e^{-\frac{x}{a}}$$ 

### Generate some Histograms

These will typically be taken from Monte Carlo.

In [4]:
h_gaus = ROOT.TH1D("h_gaus","h_gaus",100,110,160)
rand = ROOT.TRandom2()
for i in range(500):
    event = rand.Gaus(125,1.)
    h_gaus.Fill(event)

h_exp_1 = ROOT.TH1D("h_exp_1","h_exp_1",100,110,160)
for i in range(30000):
    event = rand.Exp(25)
    h_exp_1.Fill(110+event)
    
h_exp_2 = ROOT.TH1D("h_exp_2","h_exp_2",100,110,160)
for i in range(5000):
    event = rand.Exp(-25)
    h_exp_2.Fill(160+event)
    
h_gaus.SetFillColor(2)
h_exp_1.SetFillColor(4)
h_exp_2.SetFillColor(6)

Let's check that these look ok

In [5]:
leg = ROOT.TLegend(0.6,0.6,0.85,0.85)
leg.AddEntry(h_gaus, "Higgs", "f")
leg.AddEntry(h_exp_1, "Background 1", "f")
leg.AddEntry(h_exp_2, "Background 2", "f")
stack = ROOT.THStack()
stack.Add(h_exp_1)
stack.Add(h_exp_2)
stack.Add(h_gaus)
c1 = ROOT.TCanvas()
stack.Draw()
data_hist.Draw("same")
leg.Draw()
c1.Draw()

### Create Histfactory Measurement

First we set the Parameter of interest, and several constant parameters.

In [6]:
meas = ROOT.RooStats.HistFactory.Measurement("meas", "meas")
meas.SetPOI( "SigXsecOverSM" )


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



We don't include a specific Luminocity here, but since HistFactory gives it to us for free, we set it constant with a dummy value of 2%

In [7]:
meas.SetLumi( 1.0 )
meas.SetLumiRelErr( 0.02 )
meas.AddConstantParam("Lumi")

### Define a Region

Here we use the region 110 - 160 GeV to define our signal region.

#### Add data

In [8]:
chan = ROOT.RooStats.HistFactory.Channel( "SR" )
chan.SetStatErrorConfig(0.05, "Poisson")
chan.SetData( data_hist )

#### Add MC

Starting with the signal

In [9]:
signal = ROOT.RooStats.HistFactory.Sample( "signal" )
signal.SetNormalizeByTheory( False )
signal.SetHisto( h_gaus)

Add the parmaeter of interest and a systematic and try to make intelligent choice of upper bound

In [10]:
signal.AddNormFactor( "SigXsecOverSM", 1, 0, 3)

Add the normalisation uncertainty

In [11]:
uncertainty_up   = 1.1
uncertainty_down = 0.9
signal.AddOverallSys( "signal_norm_uncertainty",  uncertainty_down, uncertainty_up )

Here we create a shape uncertainty due to the width of the gaussian used

In [12]:
h_gaus_down = ROOT.TH1D("h_gaus_down","h_gaus_down",100,110,160)
h_gaus_up = ROOT.TH1D("h_gaus_up","h_gaus_up",100,110,160)
for i in range(500):
    event_down = rand.Gaus(125,.5)
    event_up = rand.Gaus(125,1.5)
    h_gaus_down.Fill(event_down)
    h_gaus_up.Fill(event_up)

This is used as part of a `HistoSys` object from the `HistFactory`

In [13]:
signal_shape = ROOT.RooStats.HistFactory.HistoSys("signal_shape")
signal_shape.SetHistoHigh( h_gaus_up )
signal_shape.SetHistoLow( h_gaus_down )

This systematic is then assigned to the sample and the sample to the channel.

In [14]:
signal.AddHistoSys( signal_shape )
chan.AddSample( signal )

#### Repeat for the background samples

In [15]:
background1 = ROOT.RooStats.HistFactory.Sample( "background1" )
background1.SetNormalizeByTheory( False )
background1.SetHisto( h_exp_1 )

In [16]:
background2 = ROOT.RooStats.HistFactory.Sample( "background2" )
background2.SetNormalizeByTheory( False )
background2.SetHisto( h_exp_2 )

In [17]:
chan.AddSample( background1 )
chan.AddSample( background2 )

The channel is now added to the measurement. 

In [18]:
meas.AddChannel(chan)

### Create a Workspace

In [19]:
hist2workspace = ROOT.RooStats.HistFactory.HistoToWorkspaceFactoryFast(meas)
workspace = hist2workspace.MakeSingleChannelModel(meas, chan)



-------------------
Starting to process SR channel with 1 observables
lumi str = [1,0,10]
lumi Error str = nominalLumi[1,0,1.2],0.02
[#1] INFO:ObjectHandling -- RooWorkspace::import(SR) importing RooStats::HistFactory::FlexibleInterpVar::signal_SR_epsilon
making normFactor: SigXsecOverSM
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(signal_SR_Hist_alphanominalDHist): fit range of variable obs_x_SR expanded to nearest bin boundaries: [110,160] --> [110,160]
Gaussian::alpha_signal_shapeConstraint(alpha_signal_shape,nom_alpha_signal_shape[0.,-10,10],1.)
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(signal_SR_Hist_alpha_0lowDHist): fit range of variable obs_x_SR expanded to nearest bin boundaries: [110,160] --> [110,160]
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(signal_SR_Hist_alpha_0highDHist): fit range of variable obs_x_SR expanded to nearest bin boundaries: [110,160] --> [110,160]
[#1] INFO:ObjectHandling -- RooWorkspace::import(SR) importing dataset signal_S

#### Save to file

In [20]:
workspace.SetName('HgammagammaWorkspace')
workspace.writeToFile("../workspaces/HgammagammaWorkspace.root")

False

## The Template

The parametrized probability density function constructed by the `HistFactory` is of a concrete form, but sufficiently flexible to describe many analyses based on template histograms. In general, the HistFactory produces probability density functions of the form
$$  
P(n_c, x_e, a_p | \phi_p, \alpha_p, \gamma_b) = \sum_{c\in\mathrm{channels}}\left[\mathrm{Pois}(n_c|\nu_c) \sum_{e=1}^{n_c} f_c(x_e|\boldsymbol{\alpha})\right] \cdot G(L_0|\lambda, \Delta L) \cdot \sum_{p\in\mathcal{S}+\boldsymbol{\Gamma}}f_p(a_p|\alpha_p)
$$
where $f_p(a_p|\alpha_p)$ is a constraint term describing an auxiliary measurement ap that constrains the nuisance parameter $\alpha_p$. Denote the bin containing $x_e$ as $b_e$. We have the following expression for the expected (mean) number of events in a given bin
$$
\nu_{cb}(\phi_p, \alpha_p, \gamma_b) = \lambda_{cs} \gamma_{cb} \phi_{cs}(\boldsymbol{\alpha}) \eta_{cs}(\boldsymbol{\alpha}) \sigma_{csb}(\boldsymbol{\alpha})
$$
where the meaning of the various terms is described below and the specific interpolation algorithms are described in [the morphing notebook](Morphing.ipynb). The mean number of events in each bin implies the following probability density
$$
f_c(x_e|\phi_p, \alpha_p, \gamma_b) = \nu_{cbe} \quad \mathrm{with}\quad \nu_c = \sum_{b\in\mathrm{bins\ of\ channel\ }c} \nu_{cb}
$$

It is perhaps more convenient to think of the likelihood as a product over 

$$  
P(n_{cb}, a_p | \phi_p, \alpha_p, \gamma_b) = \sum_{c\in\mathrm{channels}}\sum_{b\in\mathrm{bins}\mathrm{Pois}}(n_{cb}|\nu_{cb}) \cdot G(L_0|\lambda, \Delta L) \cdot \sum_{p\in\mathcal{S}+\boldsymbol{\Gamma}}f_p(a_p|\alpha_p)
$$

 * $\lambda_{cs}$ - luminosity parameter for a given channel and sample. Within a given channel this parameter is a common luminosity parameter for all the samples that include luminosity uncertainty (i.e.. `NormalizeByTheory="True"`). For all the samples with `NormalizeByTheory="False"` it is fixed to the nominal luminosity $\lambda_{cs} = L_0.$
 * $\gamma_{cb_e}$ - Bin-by-bin scale factor used for statistical uncertainties, bin-by-bin shape systematics (`ShapeSys`), and data-driven shape extrapolations (`ShapeFactor`). For statistical errors, the $\gamma_{csb_e}$ is shared for all the samples in the channel (ie. subscript s can be omitted). For samples that do not have any bin-by-bin scale factors $\gamma_{csb_e} = 1$.
 * $\phi_{cs}$ - Product of unconstrained normalization factors for a given sample within a given channel. These typically include the parameter of interest, eg. the signal cross-section or branching ratio. 
 $$
 \phi_{cs} = \sum_{p\in \mathbb{N}_c} \phi_p
 $$
 * $\eta_{cs}(\boldsymbol{\alpha})$ - The parametrized normalization uncertainties (ie. `OverallSys`) for a given sample within a given channel (a factor around 1).
 * $\sigma_{csb_e}$ - The parametrized histogram (ie. the nominal histogram and the `HistoSys`) for a given sample within a given channel.